<a href="https://colab.research.google.com/github/CACMO/demo/blob/main/Tesec_Calibracion_Agente_WDeeps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

class TesecAnalysisAgent:
    def __init__(self):
        self.initial_calibration = None
        self.adjusted_calibration = None
        self.engineer_name = None

    def load_initial_calibration(self, file_path):
        """Cargar archivo de calibración inicial."""
        try:
            self.initial_calibration = pd.read_csv(file_path)
            print("Archivo de calibración inicial cargado correctamente.")
            return True
        except Exception as e:
            print(f"Error al cargar el archivo: {e}")
            return False

    def load_adjusted_calibration(self, file_path):
        """Cargar archivo de calibración ajustada."""
        try:
            self.adjusted_calibration = pd.read_csv(file_path)
            print("Archivo de calibración ajustada cargado correctamente.")
            return True
        except Exception as e:
            print(f"Error al cargar el archivo: {e}")
            return False

    def analyze_calibration(self, data, tolerance=0.015):
        """Analizar calibración y verificar tolerancia."""
        results = []
        critical_tests = []

        for i, row in data.iterrows():
            if not pd.isna(row['values']):
                error = row['error_percentage']
                if error > tolerance:
                    critical_tests.append((i, error))
                results.append({"test_id": i, "error": error, "within_tolerance": error <= tolerance})

        critical_tests.sort(key=lambda x: x[1], reverse=True)
        return results, critical_tests[:10]

    def compare_calibrations(self, initial, adjusted, tolerance=0.015):
        """Comparar las dos calibraciones."""
        initial_results, initial_critical = self.analyze_calibration(initial, tolerance)
        adjusted_results, adjusted_critical = self.analyze_calibration(adjusted, tolerance)

        improvement = sum(1 for i, a in zip(initial_results, adjusted_results) if a['error'] < i['error'])
        average_improvement = (sum(a['error'] for a in adjusted_results) -
                              sum(i['error'] for i in initial_results)) / len(initial_results)

        return improvement, average_improvement, initial_critical, adjusted_critical

    def visualize_critical_tests(self, critical_tests, title=""):
        """Visualizar las pruebas más críticas."""
        plt.figure(figsize=(10, 6))
        plt.title(title)
        plt.bar(range(len(critical_tests)), [error for _, error in critical_tests])
        plt.xlabel("Prueba")
        plt.ylabel("Porcentaje de Error")
        plt.show()

    def generate_report(self, initial_results, adjusted_results, critical_initial, critical_adjusted):
        """Generar reporte final."""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"calibration_report_{timestamp}.txt"

        with open(filename, "w") as f:
            f.write(f"Informe de Calibración - {timestamp}\n")
            f.write(f"Ingeniero encargado: {self.engineer_name}\n\n")

            f.write("Resultados Iniciales:\n")
            for result in initial_results:
                status = "Dentro de tolerancia" if result['within_tolerance'] else "FUERA DE TOLERANCIA"
                f.write(f"Prueba {result['test_id']}: {result['error']:.2%} {status}\n")

            f.write("\nResultados Ajustados:\n")
            for result in adjusted_results:
                status = "Dentro de tolerancia" if result['within_tolerance'] else "FUERA DE TOLERANCIA"
                f.write(f"Prueba {result['test_id']}: {result['error']:.2%} {status}\n")

            f.write("\nPruebas Críticas Iniciales:\n")
            for test_id, error in critical_initial:
                f.write(f"Prueba {test_id}: {error:.2%}\n")

            f.write("\nPruebas Críticas Ajustadas:\n")
            for test_id, error in critical_adjusted:
                f.write(f"Prueba {test_id}: {error:.2%}\n")

            f.write("\nResumen:\n")
            if any(not r['within_tolerance'] for r in initial_results):
                f.write("La calibración inicial está FUERA DE TOLERANCIA.\n")
            else:
                f.write("La calibración inicial está DENTRO DE TOLERANCIA.\n")

            if any(not r['within_tolerance'] for r in adjusted_results):
                f.write("La calibración ajustada está FUERA DE TOLERANCIA.\n")
            else:
                f.write("La calibración ajustada está DENTRO DE TOLERANCIA.\n")

def main():
    agent = TesecAnalysisAgent()

    while True:
        print("\nMenú de Análisis de Calibración")
        print("1. Cargar archivo de calibración inicial")
        print("2. Analizar calibración inicial")
        print("3. Cargar archivo de calibración ajustada")
        print("4. Analizar calibración ajustada")
        print("5. Comparar calibraciones")
        print("6. Visualizar pruebas críticas")
        print("7. Generar informe final")
        print("8. Salir")

        option = input("Seleccione una opción: ")

        if option == "1":
            file_path = input("Ingrese la ruta del archivo de calibración inicial: ")
            agent.load_initial_calibration(file_path)
        elif option == "2":
            if agent.initial_calibration is not None:
                results, critical = agent.analyze_calibration(agent.initial_calibration)
                print("Resultados de la calibración inicial:")
                for result in results:
                    print(f"Prueba {result['test_id']}: {result['error']:.2%} {'dentro' if result['within_tolerance'] else 'fuera'} de tolerancia")
            else:
                print("No se ha cargado el archivo de calibración inicial.")
        elif option == "3":
            file_path = input("Ingrese la ruta del archivo de calibración ajustada: ")
            agent.load_adjusted_calibration(file_path)
        elif option == "4":
            if agent.adjusted_calibration is not None:
                results, critical = agent.analyze_calibration(agent.adjusted_calibration)
                print("Resultados de la calibración ajustada:")
                for result in results:
                    print(f"Prueba {result['test_id']}: {result['error']:.2%} {'dentro' if result['within_tolerance'] else 'fuera'} de tolerancia")
            else:
                print("No se ha cargado el archivo de calibración ajustada.")
        elif option == "5":
            if agent.initial_calibration is not None and agent.adjusted_calibration is not None:
                improvement, avg_imp, crit_initial, crit_adj = agent.compare_calibrations(agent.initial_calibration, agent.adjusted_calibration)
                print(f"Mejora promedio: {avg_imp:.2%}")
                print(f"Pruebas mejoradas: {improvement}")
                print("Pruebas críticas iniciales:")
                for test_id, error in crit_initial:
                    print(f"Prueba {test_id}: {error:.2%}")
                print("Pruebas críticas ajustadas:")
                for test_id, error in crit_adj:
                    print(f"Prueba {test_id}: {error:.2%}")
            else:
                print("Ambos archivos deben estar cargados para 비교lar.")
        elif option == "6":
            # This line needs to be fixed too: agent.visualize_critical_tests(agent.initial_calibration)
            # The `visualize_critical_tests` method expects a list of critical tests, not the entire DataFrame.
            # For now, I'm just fixing the primary syntax error.
            if agent.initial_calibration is not None:
                _, initial_critical = agent.analyze_calibration(agent.initial_calibration)
                agent.visualize_critical_tests(initial_critical, title="Pruebas Críticas Iniciales")
            else:
                print("No se ha cargado el archivo de calibración inicial.")
        elif option == "7":
            agent.engineer_name = input("Ingrese el nombre del ingeniero encargado: ")
            if agent.initial_calibration is not None and agent.adjusted_calibration is not None:
                initial_results, critical_initial = agent.analyze_calibration(agent.initial_calibration)
                adjusted_results, critical_adjusted = agent.analyze_calibration(agent.adjusted_calibration)
                agent.generate_report(initial_results, adjusted_results, critical_initial, critical_adjusted)
                print("Informe generado correctamente.")
            else:
                print("Ambos archivos de calibración deben estar cargados para generar el informe.")
        elif option == "8":
            break
        else:
            print("Opción inválida. Por favor seleccione una opción válida.")

if __name__ == "__main__":
    main()